In [3]:
import pandas as pd
import numpy as np

In [4]:
filename = 'newzy.csv'
texts = pd.read_csv(filename, sep='|').set_index('doc_id')

In [5]:
len(texts)

1026347

In [7]:
texts.head()

,doc_source,doc_title,doc_content,doc_date,doc_url
doc_id,,,,,
39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...","Washington Post50 Years on, Finding Profit in ...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
38,Google News,"Nearly 10 years later, families and officials ...","Washington PostNearly 10 years later, families...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
37,Google News,"Cuccinelli, running hard, has been hobbled in ...","Washington PostCuccinelli, running hard, has b...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,New York Daily NewsChuck Schumer endorses Hill...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
35,Google News,Sec. Kerry issues apparent warning to Iran - C...,Ahram OnlineSec. Kerry issues apparent warning...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [8]:
texts=texts.dropna()

In [9]:
texts['doc_date'] = pd.to_datetime(texts['doc_date'])
texts['year'] = texts['doc_date'].dt.year.astype(int)

In [10]:
random_seed = 111
sampled_texts = texts.sample(n=50000, random_state=random_seed)

In [11]:
sampled_texts = sampled_texts.sort_index()

In [8]:
sampled_texts.groupby('doc_source').count()

,doc_title,doc_content,doc_date,doc_url,year
doc_source,,,,,
Breitbart,510,510,510,510,510
CNN,1915,1915,1915,1915,1915
Daily Kos,641,641,641,641,641
Drudge Report,3916,3916,3916,3916,3916
Fox,2880,2880,2880,2880,2880
Google News,3310,3310,3310,3310,3310
Guardian,606,606,606,606,606
NPR,1248,1248,1248,1248,1248
New York Times,1116,1116,1116,1116,1116


In [9]:
LIB = sampled_texts[['doc_source','doc_title','doc_date', 'year', 'doc_url', 'doc_content']]

In [10]:
LIB['num_chars']=LIB['doc_content'].apply(len)

/var/folders/wp/4vny_tyn335f9k096ch7psp80000gn/T/ipykernel_75539/1511389737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LIB['num_chars']=LIB['doc_content'].apply(len)


In [11]:
LIB=LIB.drop(columns=['doc_content'])

In [12]:
LIB.head()

,doc_source,doc_title,doc_date,year,doc_url,num_chars
doc_id,,,,,,
23,Google News,Boston bombings change preparations for NYC ma...,2013-11-03,2013,http://news.google.com/news/url?sa=t&fd=R&usg=...,498
42,Google News,Kerry Says He Sees Signs That Egypt Is on Path...,2013-11-03,2013,http://news.google.com/news/url?sa=t&fd=R&usg=...,580
52,PowerLine,A Quarterback Controversy With a Political Twist?,2013-11-03,2013,http://feeds.powerlineblog.com/~r/powerlineblo...,3618
81,Guardian,Top US climate scientists support development ...,2013-11-03,2013,http://www.theguardian.com/environment/2013/no...,4372
87,Guardian,Americans won't throw the bums out in 2014 | H...,2013-11-03,2013,http://www.theguardian.com/commentisfree/2013/...,5114


In [13]:
import nltk
import warnings
warnings.filterwarnings('ignore')

In [14]:
content = sampled_texts[~sampled_texts['doc_content'].isna()]
content['doc_content'] = content['doc_content'].str.replace(r'\n', ' ').str.strip()

In [15]:
OHCO = ['doc_source', 'doc_id', 'sent_num', 'token_num']

In [16]:
content=content.reset_index().set_index(OHCO[:2])
content=content.drop(columns=['doc_date', 'doc_url', 'doc_title'])

In [17]:
SENTS = content['doc_content'].apply(lambda x: pd.Series(nltk.sent_tokenize(x))).stack().to_frame()

In [18]:
SENTS=SENTS.rename(columns={0:'sent'})

In [19]:
SENTS.head()

sent
doc_source  doc_id                                                     
Google News 23     0  NPRBoston bombings change preparations for NYC...
                   1  Play Video ...New York City Marathon returns -...
            42     0  New York TimesKerry Says He Sees Signs That Eg...
                   1  Kerry issues apparent warning to IranCBS NewsJ...
PowerLine   52     0  (John Hinderaker) Public Policy Polling ran a ...

In [20]:
TOKENS = SENTS['sent'].apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))).stack().to_frame()

In [21]:
TOKENS=TOKENS.rename(columns={0:'token_pos'})

In [22]:
TOKENS.head()

token_pos
doc_source  doc_id                         
Google News 23     0 0     (NPRBoston, NNP)
                     1      (bombings, NNS)
                     2        (change, VBP)
                     3  (preparations, NNS)
                     4            (for, IN)

In [23]:
TOKENS['pos']=TOKENS.token_pos.apply(lambda x: x[1])
TOKENS['token_str']=TOKENS.token_pos.apply(lambda x: x[0])
TOKENS=TOKENS.drop('token_pos', 1)

In [24]:
TOKENS.index.names=OHCO

In [25]:
TOKENS.head()

pos     token_str
doc_source  doc_id sent_num token_num                   
Google News 23     0        0          NNP     NPRBoston
                            1          NNS      bombings
                            2          VBP        change
                            3          NNS  preparations
                            4           IN           for

In [26]:
TOKENS['term_str'] = TOKENS['token_str'].str.lower().str.replace(r'[^a-zA-Z0-9]', '')

In [27]:
CORPUS=TOKENS.dropna().replace('',np.nan).dropna()

In [28]:
CORPUS['pos_group']=CORPUS['pos'].str[:2]

In [29]:
CORPUS

pos     token_str      term_str  \
doc_source  doc_id  sent_num token_num                                    
Google News 23      0        0          NNP     NPRBoston     nprboston   
                             1          NNS      bombings      bombings   
                             2          VBP        change        change   
                             3          NNS  preparations  preparations   
                             4           IN           for           for   
...                                     ...           ...           ...   
UPI Latest  1026343 0        13          JJ        famous        famous   
                             14         NNS     birthdays     birthdays   
                             15          IN           for           for   
                             16         NNP          Feb.           feb   
                             17          CD           27.            27   

                                       pos_group  
doc_source  doc_id  sent_num token_num            
Google News 23      0        0                NN  
                             1                NN  
                             2                VB  
                             3                NN  
                             4                IN  
...                                          ...  
UPI Latest  1026343 0        13               JJ  
                             14               NN  
                             15               IN  
                             16               NN  
                             17               CD  

[1786062 rows x 4 columns]

In [30]:
LIB

,doc_source,doc_title,doc_date,year,doc_url,num_chars
doc_id,,,,,,
23,Google News,Boston bombings change preparations for NYC ma...,2013-11-03,2013,http://news.google.com/news/url?sa=t&fd=R&usg=...,498
42,Google News,Kerry Says He Sees Signs That Egypt Is on Path...,2013-11-03,2013,http://news.google.com/news/url?sa=t&fd=R&usg=...,580
52,PowerLine,A Quarterback Controversy With a Political Twist?,2013-11-03,2013,http://feeds.powerlineblog.com/~r/powerlineblo...,3618
81,Guardian,Top US climate scientists support development ...,2013-11-03,2013,http://www.theguardian.com/environment/2013/no...,4372
87,Guardian,Americans won't throw the bums out in 2014 | H...,2013-11-03,2013,http://www.theguardian.com/commentisfree/2013/...,5114
...,...,...,...,...,...,...
1026234,US News,U.S. Obesity Rate Passes 40 Percent,2020-02-27,2020,https://www.usnews.com/news/healthiest-communi...,107
1026260,US News,Nebraska Tomato Plant Pleads Guilty in Immigra...,2020-02-27,2020,https://www.usnews.com/news/best-states/nebras...,243
1026320,Fox,Los Angeles deputies search for stolen hearse ...,2020-02-27,2020,https://www.foxnews.com/us/southern-california...,139


In [31]:
avg_chars=LIB['num_chars'].mean()

In [32]:
avg_chars

227.35344

In [33]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str', 'pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str', 'pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

In [34]:
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group
term_str,,,,,,
0,75,1,4.199182e-05,14.539532,CD,CD
00,97,2,5.430942e-05,14.168438,JJ,JJ
000,3,3,1.679673e-06,19.183388,CD,CD
0000,22,4,1.231760e-05,16.308919,NNP,NN
00001,1,5,5.598910e-07,20.768351,CD,CD


In [35]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.index.to_series().apply(stemmer.stem)

In [36]:
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group,porter_stem
term_str,,,,,,,
0,75,1,4.199182e-05,14.539532,CD,CD,0
00,97,2,5.430942e-05,14.168438,JJ,JJ,00
000,3,3,1.679673e-06,19.183388,CD,CD,000
0000,22,4,1.231760e-05,16.308919,NNP,NN,0000
00001,1,5,5.598910e-07,20.768351,CD,CD,00001


In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ryankim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
stops = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str']).reset_index().set_index('term_str')
stops.columns = ['stop']
stops['stop'] = 1

In [39]:
VOCAB['stop']=VOCAB.index.to_series().map(stops['stop'])
VOCAB['stop']=VOCAB['stop'].fillna(0).astype('int')

In [40]:
VOCAB[VOCAB.stop==1].head()

,n,n_chars,p,i,max_pos,max_pos_group,porter_stem,stop
term_str,,,,,,,,
a,53456,1,0.029930,5.062286,DT,DT,a,1
about,3915,5,0.002192,8.833554,IN,IN,about,1
above,120,5,0.000067,13.861460,IN,IN,abov,1
after,6181,5,0.003461,8.174726,IN,IN,after,1
again,644,5,0.000361,11.437434,RB,RB,again,1


In [41]:
def create_bow(CORPUS, bag, item_type='term_str'):
    BOW = CORPUS.groupby(bag+[item_type])[item_type].count().to_frame('n')
    return BOW

In [42]:
def get_tfidf(BOW, tf_method='max', df_method='standard', item_type='term_str'):
            
    DTCM = BOW.n.unstack() # Create Doc-Term Count Matrix
    
    if tf_method == 'sum':
        TF = (DTCM.T / DTCM.T.sum()).T
    elif tf_method == 'max':
        TF = (DTCM.T / DTCM.T.max()).T
    elif tf_method == 'log':
        TF = (np.log2(DTCM.T + 1)).T
    elif tf_method == 'raw':
        TF = DTCM
    elif tf_method == 'bool':
        TF = DTCM.astype('bool').astype('int')
    else:
        raise ValueError(f"TF method {tf_method} not found.")

    DF = DTCM.count() # Assumes NULLs 
    N_docs = len(DTCM)
    
    if df_method == 'standard':
        IDF = np.log2(N_docs/DF) # This what the students were asked to use
    elif df_method == 'textbook':
        IDF = np.log2(N_docs/(DF + 1))
    elif df_method == 'sklearn':
        IDF = np.log2(N_docs/DF) + 1
    elif df_method == 'sklearn_smooth':
        IDF = np.log2((N_docs + 1)/(DF + 1)) + 1
    else:
        raise ValueError(f"DF method {df_method} not found.")
    
    TFIDF = TF * IDF
    
    DFIDF = DF * IDF
    
    TFIDF = TFIDF.fillna(0)

    return TFIDF, DFIDF

In [43]:
bag=OHCO[:1]

In [44]:
BOW = create_bow(CORPUS, bag)

In [45]:
BOW

n
doc_source term_str        
Breitbart  1              2
           10             3
           100            5
           1000           2
           100000         1
...                      ..
US News    zurich         6
           zuzana         1
           zverev         1
           zverevmolkino  1
           zygmunt        1

[196722 rows x 1 columns]

In [46]:
TFIDF, DFIDF = get_tfidf(BOW)

In [47]:
VOCAB['dfidf'] = DFIDF

In [48]:
VOCAB.dfidf.sort_values(ascending=False).head(20)

term_str
watson          7.931569
reservations    7.931569
telephone       7.931569
centered        7.931569
cent            7.931569
60s             7.931569
gig             7.931569
rip             7.931569
cement          7.931569
cells           7.931569
cellphones      7.931569
temptation      7.931569
rid             7.931569
tenants         7.931569
celebrates      7.931569
richmond        7.931569
globally        7.931569
tennis          7.931569
wrongful        7.931569
tent            7.931569
Name: dfidf, dtype: float64

In [49]:
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,porter_stem,stop,dfidf
term_str,,,,,,,,,
0,75,1,4.199182e-05,14.539532,CD,CD,0,0,7.924813
00,97,2,5.430942e-05,14.168438,JJ,JJ,00,0,5.813781
000,3,3,1.679673e-06,19.183388,CD,CD,000,0,3.906891
0000,22,4,1.231760e-05,16.308919,NNP,NN,0000,0,6.965784
00001,1,5,5.598910e-07,20.768351,CD,CD,00001,0,3.906891
...,...,...,...,...,...,...,...,...,...
zverevmolkino,1,13,5.598910e-07,20.768351,NNP,NN,zverevmolkino,0,3.906891
zycher,1,6,5.598910e-07,20.768351,NNP,NN,zycher,0,3.906891
zygmunt,1,7,5.598910e-07,20.768351,NNP,NN,zygmunt,0,3.906891


In [50]:
LIB.to_csv('LIB.csv')
CORPUS.to_csv('CORPUS.csv')
VOCAB.to_csv('VOCAB.csv')

In [12]:
sampled_texts.to_csv('sampled_newzy.csv')